In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

from pydub import AudioSegment, effects
from pydub.generators import WhiteNoise
from pydub.playback import play
from pydub.utils import mediainfo
import librosa
from librosa import display   
import noisereduce as nr
import IPython.display as ipd
from IPython.display import Audio
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pytz
import cv2

import tensorflow as tf
import tensorboard
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from json_tricks import dump, load

from DataModel import DataModel
from CNNModel import CNNModel
from Evaluation import Evaluation

from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("TensorFlow version:  ", tf.__version__)
print("TensorBoard version: ", tensorboard.__version__)

tz = pytz.timezone('Asia/Hong_Kong')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


TensorFlow version:   2.7.0
TensorBoard version:  2.10.0


# Data Preparatioin

In [3]:
dataModelAll = DataModel()
dataModelAll.extractIEMOCAPData()
dataModelAll.processData()

IEMOCAP Data Extration Completed
    Number of data: 8170
      Neutral     : 1793
      Frustration : 2006
      Anger       : 1176
      Sadness     : 1266
      Happiness   : 630
      Excitement  : 1148
      Surprise    : 109
      Disgust     : 2
      Fear        : 40

Processing data to Mel Spectrogram...
Mel Spectrogram Processing Completed
    Shape of images: (8170, 256, 256, 1)

Processing labels...
Label Processing Completed

Splitting data...
Data Split Completed

Data Processing Completed!
  Data shapes:
    x_train  : (6536, 256, 256, 1)
    y_train  : (6536,)
    sr_train : (6536,)
    x_test   : (1634, 256, 256, 1)
    y_test   : (1634,)
    sr_test  : (1634,)



In [3]:
labelsToInclude = ['Neutral', 'Frustration', 'Anger', 'Sadness', 'Happiness']
mergeHappinessExcitement = True

dataModel5Labels = DataModel(labelsToInclude=labelsToInclude, mergeHappinessExcitement=mergeHappinessExcitement)
dataModel5Labels.extractIEMOCAPData()
dataModel5Labels.processData()

IEMOCAP Data Extration Completed
    Number of data: 8019
      Neutral     : 1793
      Frustration : 2006
      Anger       : 1176
      Sadness     : 1266
      Happiness   : 1778
      Excitement  : 0
      Surprise    : 0
      Disgust     : 0
      Fear        : 0

Processing data to Mel Spectrogram...
Mel Spectrogram Processing Completed
    Shape of images: (8019, 256, 256, 1)

Processing labels...
Label Processing Completed

Splitting data...
Data Split Completed

Data Processing Completed!
  Data shapes:
    x_train  : (6416, 256, 256, 1)
    y_train  : (6416,)
    sr_train : (6416,)
    x_test   : (1603, 256, 256, 1)
    y_test   : (1603,)
    sr_test  : (1603,)



In [2]:
labelsToInclude = ['Neutral', 'Frustration', 'Anger', 'Sadness', 'Happiness']
mergeHappinessExcitement = True

emoDBModel = DataModel(labelsToInclude=labelsToInclude, mergeHappinessExcitement=mergeHappinessExcitement)
emoDBModel.extractEmoDBData()
emoDBModel.processData()

EmoDB Data Extration Completed
    Number of data: 408
      Neutral     : 79
      Frustration : 69
      Anger       : 127
      Sadness     : 62
      Happiness   : 71
      Excitement  : 0
      Surprise    : 0
      Disgust     : 0
      Fear        : 0
      Boredom     : 0

Processing data to Mel Spectrogram...
Mel Spectrogram Processing Completed
    Shape of images: (408, 256, 256, 1)

Processing labels...
Label Processing Completed

Splitting data...
Data Split Completed

Data Processing Completed!
  Data shapes:
    x_train  : (327, 256, 256, 1)
    y_train  : (327,)
    sr_train : (327,)
    x_test   : (81, 256, 256, 1)
    y_test   : (81,)
    sr_test  : (81,)



In [2]:
labelsToInclude = ['Neutral', 'Frustration', 'Anger', 'Sadness', 'Happiness']
mergeHappinessExcitement = True

mixDataModel = DataModel(labelsToInclude=labelsToInclude, mergeHappinessExcitement=mergeHappinessExcitement)
mixDataModel.extractIEMOCAPData()
mixDataModel.extractEmoDBData()
mixDataModel.processData()

Data Extration Completed
    Number of data: 8427
      Neutral     : 1872
      Frustration : 2075
      Anger       : 1303
      Sadness     : 1328
      Happiness   : 1849
      Excitement  : 0
      Surprise    : 0
      Disgust     : 0
      Fear        : 0
      Boredom     : 0

Processing data to Mel Spectrogram...
Mel Spectrogram Processing Completed
    Shape of images: (8427, 256, 256, 1)

Processing labels...
Label Processing Completed

Splitting data...
Data Split Completed

Data Processing Completed!
  Data shapes:
    x_train  : (6742, 256, 256, 1)
    y_train  : (6742,)
    sr_train : (6742,)
    x_test   : (1685, 256, 256, 1)
    y_test   : (1685,)
    sr_test  : (1685,)



# Training

In [ ]:
def training1(dataModel, experimentName, modelName, epochs):
  cnnModel = CNNModel(modelName, experimentName)
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)

experimentName = "(Experiment 1) CNN Model A (1000 Epochs) (Session 1-5) (Ignore Other)"
modelName = "modelA"
epochs = 1000

training1(dataModelAll, experimentName, modelName, epochs)

In [ ]:
def training2(dataModel, experimentName, modelName, epochs):
  ySize = len(dataModel.labels_name)

  cnnModel = CNNModel(modelName, experimentName, ySize=ySize)
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

experimentName = "(Experiment 2) CNN Model A (200 Epochs) (Session 1-5) (Ignore Other) (5 Emotions with Merge)"
modelName = "modelA"
epochs = 200

training2(dataModel5Labels, experimentName, modelName, epochs)

In [ ]:
def training3(dataModel, experimentName, modelName, epochs):
  ySize = len(dataModel.labels_name)
  learning_rate = 0.00001
  decay = 0.0001

  cnnModel = CNNModel(modelName, experimentName, ySize=ySize, learning_rate=learning_rate, decay=decay)
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

experimentName = "(Experiment 3) CNN Model B (200 Epochs) (Session 1-5) (Ignore Other) (5 Emotions with Merge) (00001 lr 0001 decay)"
modelName = "modelB"
epochs = 200

training3(dataModel5Labels, experimentName, modelName, epochs)

In [ ]:
def training4(dataModel, experimentName, modelName, epochs):
  ySize = len(dataModel.labels_name)
  
  cnnModel = CNNModel(modelName, experimentName, ySize=ySize)
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

experimentName = "(Experiment 4) CNN Model C (200 Epochs) (Session 1-5) (Ignore Other) (5 Emotions with Merge)"
modelName = "modelC"
epochs = 200

training4(dataModel5Labels, experimentName, modelName, epochs)

In [ ]:
def training5(dataModel, experimentName, modelName, epochs):
  ySize = len(dataModel.labels_name)
  
  cnnModel = CNNModel(modelName, experimentName, ySize=ySize)
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

experimentName = "(Experiment 5) CNN Model D (200 Epochs) (Session 1-5) (Ignore Other) (5 Emotions with Merge)"
modelName = "modelD"
epochs = 200

training5(dataModel5Labels, experimentName, modelName, epochs)

In [ ]:
def training6(dataModel, experimentName, modelName, epochs):
  ySize = len(dataModel.labels_name)
  learning_rate = 0.00001
  decay = 0.0001

  cnnModel = CNNModel(modelName, experimentName, ySize=ySize, learning_rate=learning_rate, decay=decay)
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

experimentName = "(Experiment 6) CNN Model E (200 Epochs) (Session 1-5) (Ignore Other) (5 Emotions with Merge) (000001 lr 00001 decay)"
modelName = "modelE"
epochs = 200

training6(dataModel5Labels, experimentName, modelName, epochs)

In [ ]:
labelsToInclude = ['Neutral', 'Frustration', 'Anger', 'Sadness', 'Happiness']
mergeHappinessExcitement = True
ignoreDuration = 6

dataModel5LabelsIgnore6 = DataIEMOCAP(labelsToInclude=labelsToInclude,
                                      mergeHappinessExcitement=mergeHappinessExcitement,
                                      ignoreDuration=ignoreDuration)
dataModel5LabelsIgnore6.processData()

def training7(dataModel, experimentName, modelName, epochs):
  ySize = len(dataModel.labels_name)

  cnnModel = CNNModel(modelName, experimentName, ySize=ySize)
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

experimentName = "(Experiment 7) CNN Model A (200 Epochs) (Session 1-5) (Ignore Other) (5 Emotions with Merge and Ignore 6 lower)"
modelName = "modelA"
epochs = 200

training7(dataModel5LabelsIgnore6, experimentName, modelName, epochs)

In [ ]:
labelsToInclude = ['Neutral', 'Frustration', 'Anger', 'Sadness', 'Happiness']
mergeHappinessExcitement = True
splitDuration = 4
ignoreDuration = 1

dataModel5LabelsSplit4 = DataIEMOCAP(labelsToInclude=labelsToInclude,
                                      mergeHappinessExcitement=mergeHappinessExcitement,
                                      splitDuration=splitDuration,
                                      ignoreDuration=ignoreDuration)
dataModel5LabelsSplit4.processData()

def training8(dataModel, experimentName, modelName, epochs):
  ySize = len(dataModel.labels_name)

  cnnModel = CNNModel(modelName, experimentName, ySize=ySize)
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

experimentName = "(Experiment 8) CNN Model A (200 Epochs) (Session 1-5) (Ignore Other) (5 Emotions with Merge and Split 4)"
modelName = "modelA"
epochs = 200

training8(dataModel5LabelsSplit4, experimentName, modelName, epochs)

In [ ]:
labelsToInclude = ['Neutral', 'Frustration', 'Anger', 'Sadness', 'Happiness']
mergeHappinessExcitement = True
splitDuration = 4
ignoreDuration = 2

dataModel5LabelsSplit4Ignore2 = DataIEMOCAP(labelsToInclude=labelsToInclude,
                                      mergeHappinessExcitement=mergeHappinessExcitement,
                                      splitDuration=splitDuration,
                                      ignoreDuration=ignoreDuration)
dataModel5LabelsSplit4Ignore2.processData()

def training9(dataModel, experimentName, modelName, epochs):
  ySize = len(dataModel.labels_name)

  cnnModel = CNNModel(modelName, experimentName, ySize=ySize)
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

experimentName = "(Experiment 9) CNN Model A (200 Epochs) (Session 1-5) (Ignore Other) (5 Emotions with Merge and Split 4 Ignore 2)"
modelName = "modelA"
epochs = 200

training9(dataModel5LabelsSplit4Ignore2, experimentName, modelName, epochs)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir '/content/drive/MyDrive/HKU/FYP/Speech Emotion Recognition/'